<head>
  <meta name="author" content="Rogério de Oliveira">
  <meta institution="author" content="ITM">
</head>

<img src="https://maua.br/images/selo-60-anos-maua.svg" width=300, align="right">
<!-- <h1 align=left><font size = 6, style="color:rgb(200,0,0)"> optional title </font></h1> -->



# **P1 - RA PAR**

In [132]:
#@markdown Nome completo e RA (*\<nome\>, \<RA\>*)
Aluno = 'Rafael Rubio, 20006110' #@param {type:"string"}






In [133]:
path = 'https://github.com/Rogerio-mack/work/raw/main/data/'

In [134]:
import pandas as pd

# **Case**: Airbnb

Neste case, analise dados sobre imóveis alugados pela plataforma Airbnb em NYC no ano de 2019.


In [135]:
df = pd.read_csv(path + 'AB_NYC_2019.csv')
df.head()

id                                            name  host_id  \
0   20913              Charming 1 bed GR8 WBurg LOCATION!    79402   
1   25235               Large 2 Bedroom Great for Groups!    87773   
2   76761  Eveland the Place to Stay & Enjoy a 5-⭐️ 2bdrm   118971   
3  135465             Garden apartment close to Manhattan   663879   
4  169002                Modern Space in Charming Pre-war   805344   

     host_name neighbourhood_group       neighbourhood  latitude  longitude  \
0   Christiana            Brooklyn        Williamsburg  40.70984  -73.95775   
1       Shelly            Brooklyn  Bedford-Stuyvesant  40.68373  -73.92377   
2       Evelyn            Brooklyn         South Slope  40.66552  -73.99019   
3  Christopher            Brooklyn         Fort Greene  40.68626  -73.97598   
4         Alec           Manhattan              Harlem  40.82411  -73.94934   

         room_type  price  minimum_nights  number_of_reviews last_review  \
0  Entire home/apt    100               5                168  2018-07-22   
1  Entire home/apt    125              90                162  2019-06-28   
2  Entire home/apt    169               2                398  2019-06-28   
3  Entire home/apt    170               3                 86  2019-06-20   
4     Private room     65               2                 41  2019-06-16   

   reviews_per_month  calculated_host_listings_count  availability_365  
0               1.57                               1                 0  
1               1.46                               2               137  
2               3.97                               3               182  
3               0.91                               2               286  
4               0.43                               2                59

# Q1. Proprietários com mais imóveis na plataforma

Obtenha os 3 proprietários ("hosts") com mais imóveis sendo anunciados na plataforma, e a respectiva quantidade de imóveis anunciados.


In [136]:
top_hosts = (
    df.groupby(['host_id','host_name'])
    .size()
    .reset_index(name='num_listings')
    .sort_values(by='num_listings', ascending=False)
    .head(3)
)

top_hosts



host_id     host_name  num_listings
713  219517861  Sonder (NYC)             7
394   30283594          Kara             4
605  120762452       Stanley             4

# Q2A. Locatários profissionais $\times$ anunciantes com um único imóvel

Considere o preço médio dos imóveis. Qual a diferença do preço médio dos imóveis dos anunciantes com mais de um imóvel e dos anunciantes com um único imóvel anunciado na plataforma?

In [137]:
counts = df.groupby('host_id').size().reset_index(name='num_listings')

df_merged = pd.merge(df, counts, on='host_id', how='left')

df_more_than_one = df_merged[df_merged['num_listings'] > 1]
df_only_one = df_merged[df_merged['num_listings'] == 1]

mean_price_more_than_one = df_more_than_one['price'].mean()
mean_price_only_one = df_only_one['price'].mean()

difference = mean_price_more_than_one - mean_price_only_one

print( f'Diferença de preços: USD {difference:.2f}')



Diferença de preços: USD 67.81


# Q3A.  Manhattan *outliers*

Qual o percentual de imóveis de Manhattan com preços estatisticamente discrepantes (*outliers*) com relação aos preços de todos os imóveis?



In [138]:
df.describe()

id       host_id    latitude   longitude        price  \
count  8.040000e+02  8.040000e+02  804.000000  804.000000   804.000000   
mean   1.917309e+07  6.807031e+07   40.731820  -73.950923   153.976368   
std    1.094572e+07  7.915076e+07    0.057918    0.047799   186.643634   
min    2.091300e+04  6.889000e+04   40.548570  -74.176280     0.000000   
25%    9.775536e+06  8.546812e+06   40.690742  -73.981270    70.000000   
50%    1.969176e+07  3.010719e+07   40.723650  -73.955935   106.000000   
75%    2.923053e+07  1.068492e+08   40.764025  -73.934970   180.000000   
max    3.647217e+07  2.740800e+08   40.904840  -73.729310  2850.000000   

       minimum_nights  number_of_reviews  reviews_per_month  \
count      804.000000         804.000000         628.000000   
mean         8.106965          22.062189           1.298424   
std         20.824499          42.869156           1.483718   
min          1.000000           0.000000           0.010000   
25%          1.000000           1.000000           0.180000   
50%          3.000000           5.000000           0.675000   
75%          5.000000          21.000000           2.002500   
max        365.000000         398.000000           8.870000   

       calculated_host_listings_count  availability_365  
count                      804.000000        804.000000  
mean                         7.537313        113.261194  
std                         34.432020        130.169662  
min                          1.000000          0.000000  
25%                          1.000000          0.000000  
50%                          1.000000         49.000000  
75%                          2.000000        232.250000  
max                        327.000000        365.000000

In [139]:
Q1 = df['price'].quantile(0.25)
Q3 = df['price'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

df_manhattan = df[df['neighbourhood_group'] == 'Manhattan']

outliers_manhattan = df_manhattan[
    (df_manhattan['price'] < lower_bound) | (df_manhattan['price'] > upper_bound)
]

perc_Manhattan = (len(outliers_manhattan) / len(df_manhattan)) * 100

print( f'Percentuais de outliers de preços, Manhattan: {perc_Manhattan:.2f} %')



Percentuais de outliers de preços, Manhattan: 10.05 %


# Q4A. Preços $\times$ Descrições

Considere as descrições dos anúncios em `df.name`. Crie uma coluna `df['word']` que contém 'beautiful','sunny','spacious' ou 'heart' de acordo com a presença dessa palavra no anúncio. Caso não tenha nenhuma das palavras, assinale 'others'. Ao final, verifique a quantidade e o preço médio dos imóveis para cada grupo dessas palavras nos anúncios.

Atenção, considere a ordem 'beautiful','sunny','spacious' ou 'heart' (isto é, se o texto contém duas ou mais palavras, por exemplo 'beautiful' e 'sunny', considere 'beautiful'). Considere também quaisquer combinações de maiúsculas/minúsculas nas palavras.

In [140]:
def categorize_name(name):
    name_lower = str(name).lower()

    if 'beautiful' in name_lower:
        return 'beautiful'
    elif 'sunny' in name_lower:
        return 'sunny'
    elif 'spacious' in name_lower:
        return 'spacious'
    elif 'heart' in name_lower:
        return 'heart'
    else:
        return 'others'

df['word'] = df['name'].apply(categorize_name)

results = df.groupby('word').agg(
    count=('id','count'),
    mean_price=('price','mean')
).reset_index().sort_values(by='mean_price', ascending=True)

results





word  count  mean_price
3   spacious     60  128.516667
4      sunny     43  137.674419
0  beautiful     41  137.926829
2     others    624  157.886218
1      heart     36  166.388889

In [141]:
def categorize_name(name):
    name = str(name).lower()
    if 'beautiful' in name:
        return 'beautiful'
    elif 'sunny' in name:
        return 'sunny'
    elif 'spacious' in name:
        return 'spacious'
    elif 'heart' in name:
        return 'heart'
    else:
        return 'others'

df['word'] = df['name'].apply(categorize_name)

word_stats = df.groupby('word')['price'].agg(['count', 'mean']).sort_values(by='mean', ascending=False)

word_stats


count        mean
word                        
heart         36  166.388889
others       624  157.886218
beautiful     41  137.926829
sunny         43  137.674419
spacious      60  128.516667

In [142]:
word_stats = df.groupby('word')['price'].agg(['count', 'mean']).sort_values(by='mean', ascending=True)

word_stats

count        mean
word                        
spacious      60  128.516667
sunny         43  137.674419
beautiful     41  137.926829
others       624  157.886218
heart         36  166.388889

# Q5. Enriquecendo os dados

Enriqueça os dados dos imóveis com os valores dos impostos e multas municipais. Cacule a média de impostos e multas de cada grupo de vizinhaça, `neighbourhood_group`. Note que existem imóveis sem impostos e sem multas, esses são imóveis isentos, mas devem ser considerados nos cálculos de média (por exemplo, você pode considerar esses imóveis com valor 0).

In [143]:
df_taxes = pd.read_csv(path + 'AB_NYC_2019_taxes.csv')
df_taxes

id   type   value
0     35907127  taxes  1315.0
1       804815  taxes     NaN
2     22328585  taxes   900.0
3     27876600  taxes   869.0
4     34976037  taxes  1629.0
...        ...    ...     ...
1523  34284316  fines     NaN
1524  20325804  fines     NaN
1525  25872986  fines     NaN
1526  19365218  fines   906.0
1527   4235843  fines     NaN

[1528 rows x 3 columns]

In [144]:
df_taxes_pivot = df_taxes.pivot(index='id', columns='type', values='value').fillna(0)

print("df_taxes_pivot columns:", df_taxes_pivot.columns)

df_enriched = pd.merge(df, df_taxes_pivot, how='left', left_on='id', right_on='id')

df_enriched[['taxes', 'fines']] = df_enriched[['taxes', 'fines']].fillna(0)

avg_by_group = df_enriched.groupby('neighbourhood_group')[['taxes','fines']].mean().reset_index()

manhattan_data = avg_by_group[avg_by_group['neighbourhood_group'] == 'Manhattan']

avg_by_group




df_taxes_pivot columns: Index(['fines', 'taxes'], dtype='object', name='type')


neighbourhood_group       taxes       fines
0               Bronx  808.500000  120.150000
1            Brooklyn  798.457576  292.554545
2           Manhattan  855.029891  302.997283
3              Queens  793.987342  318.683544
4       Staten Island  992.142857  498.285714

# Q6. Quebras

Construa no `Pandas` um "relatório" (`df_report`) como abaixo. Os valores são ordenados `neighbourhood_group`,`neighbourhood`. Cada "página" contém apenas 30 linhas. A cada "quebra" de `neighbourhood_group` é gerada uma linha com preço médio do `neighbourhood_group` e, então, é iniciada uma nova página.

O item `line_id` é opcional mas deve auxiliá-lo nessa tarefa.

Ao final produza a soma dos preços das páginas abaixo...

In [145]:

df2 = df_merged.copy()

df_sorted = df2.sort_values(["neighbourhood_group", "neighbourhood"])

relatorio = []
page_id = 1
line_id = 1

groups = df_sorted.groupby("neighbourhood_group")

for neighbourhood_group, group_data in groups:
    for idx, row in group_data.iterrows():
        if line_id > 30:
            page_id += 1
            line_id = 1

        relatorio.append({
            "neighbourhood_group": row["neighbourhood_group"],
            "neighbourhood": row["neighbourhood"],
            "price": row["price"],
            "page_id": page_id,
            "line_id": line_id
        })
        line_id += 1

    mean_price = group_data["price"].mean()

    if line_id > 30:
        page_id += 1
        line_id = 1

    relatorio.append({
        "neighbourhood_group": neighbourhood_group,
        "neighbourhood": "MEAN",
        "price": mean_price,
        "page_id": page_id,
        "line_id": line_id
    })
    line_id += 1

    page_id += 1
    line_id = 1

df_report = pd.DataFrame(relatorio)

df_report.head(40)

neighbourhood_group       neighbourhood   price  page_id  line_id
0                Bronx             Belmont    29.0        1        1
1                Bronx   Claremont Village    67.0        1        2
2                Bronx           Concourse    40.0        1        3
3                Bronx            Edenwald    90.0        1        4
4                Bronx            Edenwald    80.0        1        5
5                Bronx           Fieldston    85.0        1        6
6                Bronx           Fieldston   109.0        1        7
7                Bronx             Fordham    79.0        1        8
8                Bronx          Highbridge    22.0        1        9
9                Bronx             Melrose    69.0        1       10
10               Bronx         Morris Park    40.0        1       11
11               Bronx          Mott Haven   100.0        1       12
12               Bronx         Port Morris    28.0        1       13
13               Bronx           Riverdale    99.0        1       14
14               Bronx            Van Nest   107.0        1       15
15               Bronx           Wakefield   120.0        1       16
16               Bronx           Wakefield   120.0        1       17
17               Bronx      Williamsbridge    45.0        1       18
18               Bronx      Williamsbridge    65.0        1       19
19               Bronx      Williamsbridge    36.0        1       20
20               Bronx                MEAN    71.5        1       21
21            Brooklyn           Bay Ridge   100.0        2        1
22            Brooklyn           Bay Ridge  1800.0        2        2
23            Brooklyn  Bedford-Stuyvesant   125.0        2        3
24            Brooklyn  Bedford-Stuyvesant   250.0        2        4
25            Brooklyn  Bedford-Stuyvesant    59.0        2        5
26            Brooklyn  Bedford-Stuyvesant    45.0        2        6
27            Brooklyn  Bedford-Stuyvesant    55.0        2        7
28            Brooklyn  Bedford-Stuyvesant    57.0        2        8
29            Brooklyn  Bedford-Stuyvesant    59.0        2        9
30            Brooklyn  Bedford-Stuyvesant   150.0        2       10
31            Brooklyn  Bedford-Stuyvesant    60.0        2       11
32            Brooklyn  Bedford-Stuyvesant   100.0        2       12
33            Brooklyn  Bedford-Stuyvesant    50.0        2       13
34            Brooklyn  Bedford-Stuyvesant   150.0        2       14
35            Brooklyn  Bedford-Stuyvesant    70.0        2       15
36            Brooklyn  Bedford-Stuyvesant    65.0        2       16
37            Brooklyn  Bedford-Stuyvesant    45.0        2       17
38            Brooklyn  Bedford-Stuyvesant    70.0        2       18
39            Brooklyn  Bedford-Stuyvesant    90.0        2       19

In [146]:
sum_price_pages = df_report[ df_report.page_id.isin([1,24,25,28]) ].groupby('page_id').price.sum().astype(int).sum()

print(f'A soma dos preços das páginas 1, 24, 25 e 28 é: {sum_price_pages}')

A soma dos preços das páginas 1, 24, 25 e 28 é: 12633


#### Aqui um *sample* do report.

In [147]:
df_report.iloc[-10::,:]

neighbourhood_group  neighbourhood       price  page_id  line_id
799              Queens       Woodside  120.000000       29       19
800              Queens           MEAN   92.227848       29       20
801       Staten Island  Arden Heights   83.000000       30        1
802       Staten Island  Randall Manor   99.000000       30        2
803       Staten Island    South Beach   20.000000       30        3
804       Staten Island    South Beach  100.000000       30        4
805       Staten Island     St. George  130.000000       30        5
806       Staten Island      Todt Hill  135.000000       30        6
807       Staten Island  Tompkinsville   59.000000       30        7
808       Staten Island           MEAN   89.428571       30        8



Seleção dos dados: Determinar quais tabelas, colunas ou variáveis serão mantidas e quais serão descartadas com base nos objetivos.

Tratamento de Missing values: Identificar valores nulos ou inválidos como lidar com eles, seja removendo linhas, substituindo valores por média/mediana ou outros

Correção de inconsistências e limpeza: Verificar duplicidades, erros de digitação ou formatação, unidades incorretas etc. e padronizar as informações.

Transformação de dados: Criar novas variáveis ou combinar atributos existentes para melhorar a capacidade de extrair padrões.

Tratamento de outliers: Identificar valores muito discrepantes (outliers) e decidir se serão corrigidos, removidos ou transformados.

Integração/União de fontes: Combinar dados de diferentes fontes (base principal e dados externos ou complementares), garantindo consistência no uso de chaves de integração e padronizando formatos.